In [1]:

from ipywidgets import interact, interact_manual, fixed, IntSlider, FloatSlider

import numpy as np
import scipy
from scipy import stats, optimize, interpolate
import matplotlib.pyplot as plt
def trunc_laplace(support, scale, gen=np.random, size=1):
    """
    Generate laplacian with support [-support, +support], and scale=scale
    """
    return ((-1) ** np.random.randint(2, size=size)) * scipy.stats.truncexpon.rvs(b=support / scale, scale=scale, size=size)

def compute_stats(m, epsilon, delta=None, Delta=2):
    if delta is None:
        delta = 1/m
    s = Delta / epsilon * np.log(m * (np.exp(epsilon) - 1) / delta + 1)
    return s, (Delta / epsilon)

def perturb(m, epsilon, delta=None, Delta=2):
    if delta is None:
        delta = 1/m
    s = Delta / epsilon * np.log(m * (np.exp(epsilon) - 1) / delta + 1)
    eta = trunc_laplace(support=s, scale=Delta / epsilon)
    # Test here?
    # b = min(1, s - eta)
    return s - eta

def trunc_laplace_pdf(x, support, loc, scale):
    return np.exp(-abs((x-loc)/scale)) / (2 * scale * (1 - np.exp(-support / scale)))

num_samples = 1000
def sample_draw(m, epsilon, num_samples=1000):
    noise_mean, noise_scale = compute_stats(m, epsilon)
#     print(f"Mean: {noise_mean}")
#     print(f"Scale: {noise_scale}")
    samples = np.array([perturb(m, epsilon) for _ in range(num_samples)])
    fig, ax = plt.subplots(1, 1, figsize=(10,5))

    x = np.linspace(0, 2 * noise_mean, num_samples)
    ax.set_title(f"Truncated Laplacian Noise (m={m}, epsilon={epsilon}, samples={num_samples})")
    ax.plot(x, trunc_laplace_pdf(x, noise_mean, noise_mean, noise_scale),
            'r-', lw=1, alpha=0.6, label=f'trunc_laplace(radius={noise_mean:.1f}, scale={noise_scale:.1f})')

    ax.hist(samples, density=True, histtype='stepfilled', alpha=0.2, bins=50)
    ax.legend(loc='best', frameon=False)
    plt.show()

In [2]:
m_slider = IntSlider(min=1, max=100, step=1, value=1)
eps_slider = FloatSlider(min=0.01, max=10, step=0.01, value=1)
num_slider = IntSlider(min=1, max=10000, step=10, value=100)
interact_manual(sample_draw, m=m_slider, epsilon=eps_slider, num_samples=num_slider)

interactive(children=(IntSlider(value=1, description='m', min=1), FloatSlider(value=1.0, description='epsilon'…

<function __main__.sample_draw(m, epsilon, num_samples=1000)>

In [3]:
m_slider = IntSlider(min=1, max=100, step=1, value=1)
eps_slider = FloatSlider(min=0.01, max=10, step=0.01, value=1)
num_slider = IntSlider(min=1, max=500, step=10, value=100)
interact(sample_draw, m=m_slider, epsilon=eps_slider, num_samples=num_slider, continuous_update=False)

interactive(children=(IntSlider(value=1, description='m', min=1), FloatSlider(value=1.0, description='epsilon'…

<function __main__.sample_draw(m, epsilon, num_samples=1000)>